In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import keras.callbacks
import tensorflow as tf
from keras.layers import Dense, Input, GlobalMaxPool1D
from keras.layers import TimeDistributed, Bidirectional, LSTM, Lambda, concatenate
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from keras import backend as K
from sklearn.metrics import classification_report

In [0]:
train_data = '/content/drive/My Drive/train.csv'
test_data = '/content/drive/My Drive/test.csv'
train_df = pd.read_csv(train_data,delimiter='\t')
test_df = pd.read_csv(test_data,delimiter = '\t')
train_df.head()

,meta,uid,sentiment
0,meta,3,negative
1,@,O,NaN
2,AdilNisarButt,Hin,NaN
3,pakistan,Hin,NaN
4,ka,Hin,NaN


In [0]:
test_df.head()

,meta,uid,sentiment
0,meta,8,neutral
1,RT,Eng,NaN
2,@,O,NaN
3,UAAPconfessions,Eng,NaN
4,Love,Eng,NaN


In [0]:
str1 = ""
tweets_train = []
sentiments_train = []
a = train_df.loc[:,['meta','sentiment']]
loops = a.shape[0]
t = ""
for i in range (loops):
  if(a.iloc[i,0] == "meta"):
    if (i!=0):
      tweets_train.append(t)
    sentiments_train.append(str(a.iloc[i,1]))
    t = ""
  else:
    t+=str(a.iloc[i,0])
    t+=" "
tweets_train.append(t)
print(tweets_train[0])
print(sentiments_train[0])

@ AdilNisarButt pakistan ka ghra tauq he Pakistan Israel ko tasleem nahein kerta Isko Palestine kehta he - OCCUPIED PALESTINE 
negative


In [0]:
str1 = ""
tweets_test = []
sentiments_test = []
a = test_df.loc[:,['meta','sentiment']]
loops = a.shape[0]
t = ""
for i in range (loops):
  if(a.iloc[i,0] == "meta"):
    if (i!=0):
      tweets_test.append(t)
    sentiments_test.append(str(a.iloc[i,1]))
    t = ""
  else:
    t+=str(a.iloc[i,0])
    t+=" "
tweets_test.append(t)
print(tweets_test[0])
print(sentiments_test[0])

RT @ UAAPconfessions Love looks good on Maddie !!! Ako lang ba yung sobrang masaya kasi may zolo sya ? Before with the past Z medyo lowkey s … 
neutral


In [0]:
tweets_train = [i.lower() for i in tweets_train]
tweets_test = [i.lower() for i in tweets_test]

tokenizer = Tokenizer(char_level = True, oov_token = "U")
tokenizer.fit_on_texts(tweets_train)

tweets_train_sequences = tokenizer.texts_to_sequences(tweets_train)
tweets_test_sequences = tokenizer.texts_to_sequences(tweets_test)

train_data = pad_sequences(tweets_train_sequences,maxlen = 150,padding = "post")
test_data = pad_sequences(tweets_test_sequences,maxlen = 150,padding = "post")

In [0]:
train_list = []
test_list = []

for i in sentiments_train:
  if i == "positive":
    train_list.append(0)
  elif i == "negative":
    train_list.append(2)
  else:
    train_list.append(1)

train_list = np.array(train_list,dtype = 'float32')

for i in sentiments_test:
  if i == "positive":
    test_list.append(0)
  elif i == "negative":
    test_list.append(2)
  else:
    test_list.append(1)

test_list = np.array(test_list,dtype = 'float32')

#convert to np.array

train_data = np.array(train_data)
test_data = np.array(test_data)

train_list = to_categorical(train_list)
test_list = to_categorical(test_list)

In [0]:
inp_size = 150
v_size = len(tokenizer.word_index) #900
no_classes = 3
dropout = 0.5

weights = []
weights.append(np.zeros(900))
for _,i in tokenizer.word_index.items():
  t = np.zeros(v_size)
  t[i-1] = 1
  weights.append(t)
weights = np.array(weights)
weights.shape

(901, 900)

In [0]:
embedding = Embedding(v_size+1,900,weights = [weights])
input1 = Input(shape = (inp_size,),dtype = 'int64') # (?,150)

layer = embedding(input1)
# Convolution 1D and MaxPooling 1D

layer = Conv1D(512,6)(layer)
layer = Activation('relu')(layer)
layer = MaxPooling1D(3)(layer)

layer = Conv1D(512,3)(layer)
layer = Activation('relu')(layer)
layer = MaxPooling1D(2)(layer)

layer = Conv1D(512,3)(layer)
layer = Activation('relu')(layer)
#layer = MaxPooling1D(2)(layer)

layer = Conv1D(256,3)(layer)
layer = Activation('relu')(layer)
#layer = MaxPooling1D(2)(layer)

layer = Conv1D(256,3)(layer)
layer = Activation('relu')(layer)

layer = Conv1D(256,3)(layer)
layer = Activation('relu')(layer)
layer = MaxPooling1D(2)(layer)

layer = Flatten()(layer) #(?,1792)

layer = Dense(1024,activation = "relu")(layer)
layer = Dropout(dropout)(layer)
layer = Dense(1024,activation="relu")(layer)
layer = Dropout(dropout)(layer) #(?,1024)

final = Dense(3,activation = "softmax")(layer)

model = Model(inputs = input1,outputs = final)

model.compile(optimizer = "adam",loss = "categorical_crossentropy",metrics = ['accuracy'])

model.summary()












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 900)          810900    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 145, 512)          2765312   
_________________________________________________________________
activation_1 (Activation)    (None, 145, 512)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 48, 512)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 512)           786944  

In [0]:
model.fit(train_data,train_list,validation_data=(test_data,test_list),batch_size=128,epochs = 25,verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 13524 samples, validate on 1869 samples
Epoch 1/25
13524/13524 [==============================] - 26s 2ms/step - loss: 1.0900 - acc: 0.3644 - val_loss: 1.0517 - val_acc: 0.4259
Epoch 2/25
13524/13524 [==============================] - 21s 2ms/step - loss: 1.0016 - acc: 0.4686 - val_loss: 1.0207 - val_acc: 0.4612
Epoch 3/25
13524/13524 [==============================] - 21s 2ms/step - loss: 0.9112 - acc: 0.5448 - val_loss: 1.0192 - val_acc: 0.5072
Epoch 4/25
13524/13524 [==============================] - 21s 2ms/step - loss: 0.8587 - acc: 0.5902 - val_loss: 0.9759 - val_acc: 0.5276
Epoch 5/25
13524/13524 [==============================] - 21s 2ms/step - loss: 0.7972 - acc: 0.6330 - val_loss: 1.1191 - val_acc: 0.5094
Epoch 6/25
13524/13524 [==============================] - 21s 2ms/step - loss: 0.7467 - acc: 0.6588 - val_loss: 1.0755 - val_acc: 0.5393
Epoch 7/25
13524/13524 [=========

In [0]:
y_pred = model.predict(test_data, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
#print(y_pred_bool)
test_list1 = np.argmax(test_list,axis = 1)
print(classification_report(test_list1, y_pred_bool))
#Here 0 is negative, 1 is neutral and 2 is positive

1869/1869 [==============================] - 1s 706us/step
              precision    recall  f1-score   support

           0       0.56      0.48      0.52       582
           1       0.48      0.61      0.54       754
           2       0.58      0.44      0.50       533

    accuracy                           0.52      1869
   macro avg       0.54      0.51      0.52      1869
weighted avg       0.53      0.52      0.52      1869

